In [14]:
import pandas as pd
import numpy as np
import re
from skopt import BayesSearchCV
from skopt.space import Real, Integer
import xgboost as xgb
from sklearn.metrics import confusion_matrix,accuracy_score, ConfusionMatrixDisplay, roc_auc_score, roc_curve
from sklearn.model_selection import train_test_split, StratifiedKFold, cross_val_score
from sklearn.multiclass import OneVsRestClassifier
import sys
sys.path.append('../src')
from gutatlas.features import clean_feature_names
from tensorflow import keras
import matplotlib.pyplot as plt
import shap
import os
import json
from pathlib import Path
os.environ['CUDA_VISIBLE_DEVICES'] = '-1'

In [5]:
training_set = pd.read_parquet('../data/processed/gi_multilabel_training.parquet')
training_set = training_set.rename({col:clean_feature_names(col) for col in training_set.columns}, axis = 1)
X_train,X_test,y_train,y_test = train_test_split(training_set.iloc[:,13:],training_set.iloc[:,1:13])

## one vs rest xgboost

In [13]:
bst = xgb.XGBClassifier(
    n_estimators=400, 
    max_depth=5, 
    learning_rate=0.08, 
    objective='binary:logistic',
    eval_metric = 'auc',
    tree_method = 'hist',
    )

multilabel_bst = OneVsRestClassifier(bst)
multilabel_bst.fit(X_train,y_train)
preds = multilabel_bst.predict(X_test)

## neural net

model = k